In [2]:
import pandas as pd
currency_table = pd.read_html('https://www.xe.com/symbols.php')
symbols = currency_table[0][1][1:]
print(symbols)

1      ALL
2      AFN
3      ARS
4      AWG
5      AUD
      ... 
105    UZS
106    VEF
107    VND
108    YER
109    ZWD
Name: 1, Length: 109, dtype: object


In [3]:
import pandas as pd

ticker_table = pd.read_html('https://coinranking.com/')
tickers = ticker_table[0]['Cryptocurrency'].dropna()
tickers = tickers.apply(lambda x: str(x).split()[-1]) #top 50 cryptocurrency tickers, mapping obtained
tickers = tickers.append(symbols)
missing = pd.Series(['UGX', 'XAU', 'ZMW', 'BITCNY', 'AED', 'pDOTn', 'BCHABC', 'UNIs', 'KCS', 'ICP', 'DREP'])
tickers = tickers.append(missing)
tickers = tickers.reset_index()
tickers = tickers.rename(columns = {0: 'Currencies'})
print(tickers)

     index Currencies
0        0        BTC
1        1        ETH
2        2       USDT
3        3        BNB
4        4        ADA
..     ...        ...
165      6     BCHABC
166      7       UNIs
167      8        KCS
168      9        ICP
169     10       DREP

[170 rows x 2 columns]


In [4]:
#graph intialization
import numpy as np
adj_matrix = np.ndarray(shape = (len(tickers) + 1, len(tickers) + 1))
#node 170 is the dummy start, additional dummy node for shortest path.
for i in range(len(tickers) + 1):
    for j in range(len(tickers) + 1):
        adj_matrix[i, j] = -1
        if i == 170:
            adj_matrix[i, j] = 1
print(adj_matrix)

[[-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 ...
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [ 1.  1.  1. ...  1.  1.  1.]]


In [15]:
from datetime import datetime

def BellmanFord(adj_matrix):
    edges = [] #remove (0, 0)
    for i in range(len(adj_matrix)):
        for j in range(len(adj_matrix)):
            if adj_matrix[i, j] != -1:
                adj_matrix[i, j] = -1 * np.log(adj_matrix[i, j])
                if i != j:
                    edges.append((i, j))
            #also, modify every edge weight
    distances = np.ndarray(shape = len(adj_matrix))
    parents = np.ndarray(shape = len(adj_matrix))
    print(adj_matrix)
    print(edges)
    for i in range(len(adj_matrix)):
        distances[i] = 9999999999999999
        parents[i] = -1
    distances[len(adj_matrix) - 1] = 0
    for i in range(len(adj_matrix) - 1):
        for edge in edges:
            if distances[edge[1]] > distances[edge[0]] + adj_matrix[edge[0], edge[1]]:
                distances[edge[1]] = distances[edge[0]] + adj_matrix[edge[0], edge[1]]
                parents[edge[1]] = edge[0]
    num_arbitrage = 0 
    print(parents) #why does 4 never appear? maybe cuz of neg cycle.
    for edge in edges:
        if distances[edge[1]] > distances[edge[0]] + adj_matrix[edge[0], edge[1]]:
            #negative weight cycle
            num_arbitrage += 1
            print("arbitrage detected!")
            #flawed logic -- think about how to detect cycle path. Draw out given shortest path tree
            # -> parents array is hard to use because of multiple possible arbitrage paths

            # num_arbitrage += 1
            # parent = edge[0]
            # arbitrage_path = [edge[1]]
            # profit_pct = 1
            # while parent != len(adj_matrix) - 1: 
            #     arbitrage_path.append(parent)
            #     length = len(arbitrage_path)
            #     frm = int(arbitrage_path[length - 2])
            #     to = int(arbitrage_path[length - 1])
            #     profit_pct *= np.exp(adj_matrix[frm, to])
            #     parent = parents[int(parent)]
            # arbitrage_path.append(edge[1])
            # length = len(arbitrage_path)
            # profit_pct *= np.exp(adj_matrix[int(arbitrage_path[length - 1]), edge[1]])
            # arbitrage_path.reverse()
            # print('arbitrage found at ' + str(datetime.now()))
            # print(arbitrage_path)
            # print('return of ' + str(profit_pct))

    if num_arbitrage == 0:
        print('no arbitrage found at ' + str(datetime.now())) 
    return


In [16]:
#BF testing
matrix = np.array([[0.8, 0.7, 0.5, 0.2, -1], [0.9, 1, 0.6, 0.8, -1], [0.3, 0.5, 1, 0.7, -1], [0.2, 0.1, 0.9, 1, -1], [1, 1, 1, 1, 1]]) #no arbitrage
BellmanFord(matrix)

[[ 0.22314355  0.35667494  0.69314718  1.60943791 -1.        ]
 [ 0.10536052 -0.          0.51082562  0.22314355 -1.        ]
 [ 1.2039728   0.69314718 -0.          0.35667494 -1.        ]
 [ 1.60943791  2.30258509  0.10536052 -0.         -1.        ]
 [-0.         -0.         -0.         -0.         -0.        ]]
[(0, 1), (0, 2), (0, 3), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (4, 3)]
[ 4.  4.  4.  4. -1.]
no arbitrage found at 2021-07-31 19:53:02.548432


In [17]:
#BF testing
matrix = np.array([[1, 1.3, 0.5, 0.2, -1], [0.9, 1, 1.1, 0.8, -1], [0.3, 0.5, 1, 1.7, -1], [3, 0.1, 0.9, 1, -1], [1, 1, 1, 1, 1]]) #1234 arbitrage
BellmanFord(matrix)

[[-0.         -0.26236426  0.69314718  1.60943791 -1.        ]
 [ 0.10536052 -0.         -0.09531018  0.22314355 -1.        ]
 [ 1.2039728   0.69314718 -0.         -0.53062825 -1.        ]
 [-1.09861229  2.30258509  0.10536052 -0.         -1.        ]
 [-0.         -0.         -0.         -0.         -0.        ]]
[(0, 1), (0, 2), (0, 3), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (4, 3)]
[ 3.  0.  3.  2. -1.]
arbitrage detected!
arbitrage detected!
arbitrage detected!


In [18]:
import requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'text/plain, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest'
}
ex_rates = pd.read_html(requests.get('https://www.investing.com/crypto/currency-pairs', headers = headers).text)
for table in ex_rates:
    relevant = table[['Name', 'Last']]
    for index, row in relevant.iterrows():
        from_coin = row['Name'].split('/')[0]
        try:
            to_coin = row['Name'].split('/')[1]
        except:
            continue #name is not an exchange pair
        try:
            from_idx = int(tickers[tickers['Currencies'] == from_coin]['index'])
            to_idx = int(tickers[tickers['Currencies'] == to_coin]['index'])
        except:
            continue 
        adj_matrix[from_idx, to_idx] = max(row['Last'], adj_matrix[from_idx, to_idx]) #in case repeated quote but other exchange has better rate

BellmanFord(adj_matrix) #remains to run Bellman-Ford on this matrix - if too long, drop some currencies that are not relevant

HT
LUNAt
LUNAt
LUNAt
LUNAt
LUNAt
ICP
ICP
ICP
ICP
PRIV
PRIV
[[-18.75589505  -3.13717975 -13.86007466 ...  -1.          -1.
   -1.        ]
 [ -1.02342882  -0.35149455  -1.         ...  -1.          -1.
   -1.        ]
 [ -1.          -1.          -1.         ...  -1.          -1.
   -1.        ]
 ...
 [ -1.          -1.          -1.         ...  -1.          -1.
   -1.        ]
 [ -1.          -1.          -1.         ...  -1.          -1.
   -1.        ]
 [         inf          inf          inf ...          inf          inf
           inf]]
[(0, 1), (0, 2), (0, 3), (0, 5), (0, 16), (0, 19), (0, 22), (0, 33), (0, 42), (0, 45), (0, 46), (0, 49), (0, 51), (0, 55), (0, 61), (0, 63), (0, 69), (0, 71), (0, 79), (0, 82), (0, 84), (0, 87), (0, 90), (0, 92), (0, 93), (0, 97), (0, 99), (0, 101), (0, 102), (0, 103), (0, 107), (1, 0), (1, 5), (1, 16), (1, 19), (1, 22), (1, 25), (1, 33), (1, 42), (1, 45), (1, 46), (1, 49), (1, 51), (1, 55), (1, 61), (1, 63), (1, 71), (1, 78), (1, 79), (1, 82), (1, 

<ipython-input-15-0548c6f8c11e>:8: RuntimeWarning: divide by zero encountered in log
  adj_matrix[i, j] = -1 * np.log(adj_matrix[i, j])
